<a href="https://colab.research.google.com/github/TheAmirHK/Experiments/blob/main/FederatedLearning/FederatedLearning_HospitalUsecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [17]:
# Here I asked ChatGPT to simulate random data. The data can be replaced by real data in this part !i!i

# Simulate healthcare datasets for 5 hospitals
def generate_data(num_samples):
    # Features: age, BMI, blood pressure, cholesterol levels, glucose levels
    X = np.random.rand(num_samples, 5) * 100  # Random values between 0 and 100
    # Labels: 1 for readmission, 0 for no readmission
    y = (X[:, 0] * 0.3 + X[:, 1] * 0.2 + X[:, 2] * 0.1 > 50).astype(int)
    return X, y

# Create datasets for 5 hospitals
hospital_data = [generate_data(1000) for _ in range(5)]

In [18]:
# Convert datasets to PyTorch DataLoader objects
def create_dataloader(X, y, batch_size=32):
    tensor_X = torch.FloatTensor(X)
    tensor_y = torch.LongTensor(y)
    dataset = TensorDataset(tensor_X, tensor_y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

hospital_loaders = [create_dataloader(X, y) for X, y in hospital_data]

In [19]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        return self.fc(x)

In [20]:
def train_local_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model

In [21]:
def federated_learning(hospital_loaders, global_model, rounds=15, local_epochs=5):
    criterion = nn.CrossEntropyLoss()
    global_optimizer = optim.SGD(global_model.parameters(), lr=0.01)

    for round in range(rounds):
        print(f"Round {round + 1}/{rounds}")
        local_weights = []

        # local training
        for dataloader in hospital_loaders:
            local_model = SimpleNN(input_dim=5)
            local_model.load_state_dict(global_model.state_dict())
            local_optimizer = optim.SGD(local_model.parameters(), lr=0.001)

            trained_model = train_local_model(local_model, dataloader, criterion, local_optimizer, epochs=local_epochs)
            local_weights.append(trained_model.state_dict())

        # Aggregate weights to be mitigated into the global model !
        new_weights = {key: torch.stack([weights[key] for weights in local_weights], dim=0).mean(dim=0)
                       for key in local_weights[0].keys()}

        # Update global model
        global_model.load_state_dict(new_weights)

    return global_model

In [22]:
def evaluate_model(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

X_test, y_test = generate_data(500)
test_loader = create_dataloader(X_test, y_test)

global_model = SimpleNN(input_dim=5)

trained_global_model = federated_learning(hospital_loaders, global_model, rounds=10, local_epochs=5)

accuracy = evaluate_model(trained_global_model, test_loader)
print(f"Global Model Accuracy: {accuracy * 100:.2f}%")

Round 1/10
Round 2/10
Round 3/10
Round 4/10
Round 5/10
Round 6/10
Round 7/10
Round 8/10
Round 9/10
Round 10/10
Global Model Accuracy: 95.40%
